<a href="https://colab.research.google.com/github/juma-paul/deep_learning/blob/main/LinearRegressionAsASingleNeuronNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install scikeras tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasRegressor
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path_car_price_dataset = '/content/drive/MyDrive/Datasets/car_price.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv(file_path_car_price_dataset)

In [ ]:
data

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


In [ ]:
data.columns

Index(['car_ID', 'symboling', 'CarName', 'fueltype', 'aspiration',
       'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'wheelbase',
       'carlength', 'carwidth', 'carheight', 'curbweight', 'enginetype',
       'cylindernumber', 'enginesize', 'fuelsystem', 'boreratio', 'stroke',
       'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg',
       'price'],
      dtype='object')

In [ ]:
data.rename(columns = {
    'car_ID': 'car_id',
    'symboling': 'safety_rating',
    'CarName': 'car_name',
    'fueltype': 'fuel_type',
    'aspiration': 'aspiration_type',
    'doornumber': 'door_number',
    'carbody': 'car_body_style',
    'drivewheel': 'drive_wheel_type',
    'enginelocation': 'engine_location',
    'wheelbase': 'wheel_base',
    'carlength': 'car_length',
    'carwidth': 'car_width',
    'carheight': 'car_height',
    'curbweight': 'curb_weight',
    'enginetype': 'engine_type',
    'cylindernumber': 'cylinder_number',
    'enginesize': 'engine_size',
    'fuelsystem': 'fuel_system',
    'boreratio': 'bore_ratio',
    'stroke': 'stroke_length',
    'compressionratio': 'compression_ratio',
    'horsepower': 'horse_power',
    'peakrpm': 'peak_rpm',
    'citympg': 'city_mpg',
    'highwaympg': 'highway_mpg',
    'price': 'price'
}, inplace=True)

In [ ]:
data = data[['safety_rating', 'fuel_type', 'aspiration_type', 'car_body_style', 'drive_wheel_type', 'engine_location', 'car_length',
      'car_width', 'curb_weight', 'cylinder_number', 'engine_size', 'bore_ratio', 'compression_ratio', 'horse_power', 'peak_rpm',
      'city_mpg', 'highway_mpg', 'price']].copy()

In [ ]:
data.head()

,safety_rating,fuel_type,aspiration_type,car_body_style,drive_wheel_type,engine_location,car_length,car_width,curb_weight,cylinder_number,engine_size,bore_ratio,compression_ratio,horse_power,peak_rpm,city_mpg,highway_mpg,price
0,3,gas,std,convertible,rwd,front,168.8,64.1,2548,four,130,3.47,9.0,111,5000,21,27,13495.0
1,3,gas,std,convertible,rwd,front,168.8,64.1,2548,four,130,3.47,9.0,111,5000,21,27,16500.0
2,1,gas,std,hatchback,rwd,front,171.2,65.5,2823,six,152,2.68,9.0,154,5000,19,26,16500.0
3,2,gas,std,sedan,fwd,front,176.6,66.2,2337,four,109,3.19,10.0,102,5500,24,30,13950.0
4,2,gas,std,sedan,4wd,front,176.6,66.4,2824,five,136,3.19,8.0,115,5500,18,22,17450.0


In [ ]:
data.dtypes

,0
safety_rating,int64
fuel_type,object
aspiration_type,object
car_body_style,object
drive_wheel_type,object
engine_location,object
car_length,float64
car_width,float64
curb_weight,int64
cylinder_number,object


In [ ]:
data.duplicated().sum()

2

In [ ]:
data = data[data.duplicated(keep='first') | ~data.duplicated(keep=False)]
data.duplicated().sum()

0

In [ ]:
data.isnull().sum()

,0
safety_rating,0
fuel_type,0
aspiration_type,0
car_body_style,0
drive_wheel_type,0
engine_location,0
car_length,0
car_width,0
curb_weight,0
cylinder_number,0


In [ ]:
X = data.drop('price', axis=1)
y = data['price']

In [ ]:
X

,safety_rating,fuel_type,aspiration_type,car_body_style,drive_wheel_type,engine_location,car_length,car_width,curb_weight,cylinder_number,engine_size,bore_ratio,compression_ratio,horse_power,peak_rpm,city_mpg,highway_mpg
0,3,gas,std,convertible,rwd,front,168.8,64.1,2548,four,130,3.47,9.0,111,5000,21,27
1,3,gas,std,convertible,rwd,front,168.8,64.1,2548,four,130,3.47,9.0,111,5000,21,27
2,1,gas,std,hatchback,rwd,front,171.2,65.5,2823,six,152,2.68,9.0,154,5000,19,26
3,2,gas,std,sedan,fwd,front,176.6,66.2,2337,four,109,3.19,10.0,102,5500,24,30
4,2,gas,std,sedan,4wd,front,176.6,66.4,2824,five,136,3.19,8.0,115,5500,18,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,gas,std,sedan,rwd,front,188.8,68.9,2952,four,141,3.78,9.5,114,5400,23,28
201,-1,gas,turbo,sedan,rwd,front,188.8,68.8,3049,four,141,3.78,8.7,160,5300,19,25
202,-1,gas,std,sedan,rwd,front,188.8,68.9,3012,six,173,3.58,8.8,134,5500,18,23
203,-1,diesel,turbo,sedan,rwd,front,188.8,68.9,3217,six,145,3.01,23.0,106,4800,26,27


In [ ]:
y

,price
0,13495.0
1,16500.0
2,16500.0
3,13950.0
4,17450.0
...,...
200,16845.0
201,19045.0
202,21485.0
203,22470.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [ ]:
numeric_data = X_train.select_dtypes(include=['int64', 'float64']).columns
numeric_data

Index(['safety_rating', 'car_length', 'car_width', 'curb_weight',
       'engine_size', 'bore_ratio', 'compression_ratio', 'horse_power',
       'peak_rpm', 'city_mpg', 'highway_mpg'],
      dtype='object')

In [ ]:
categorical_data = X_train.select_dtypes(include=['object']).columns
categorical_data

Index(['fuel_type', 'aspiration_type', 'car_body_style', 'drive_wheel_type',
       'engine_location', 'cylinder_number'],
      dtype='object')

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
   ('imputer', SimpleImputer(strategy='most_frequent')),
   ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_data),
        ('cat', categorical_transformer, categorical_data)
    ]
)

In [ ]:
# X_train_transformed = preprocessor.fit_transform(X_train)

In [64]:
def create_model(learning_rate=0.01):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(1)
    ])

    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
        loss='mean_squared_error'
    )

    return model

model = KerasRegressor(build_fn=create_model, learning_rate=0.01, verbose=0)

In [65]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model)
])

In [66]:
param_grid = {
    'regressor__batch_size': [16, 32],
    'regressor__epochs': [500, 1000],
}

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
best_model.fit(X_train, y_train)

In [ ]:
test_loss = model.evaluate(X_test, y_test, verbose=0)

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
print(f"Best model parameters: {grid_search.best_params_}")
print(f"Accuracy on test set: {test_loss:.4f}")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path_insurance_dataset = '/content/drive/MyDrive/Datasets/InsuranceData.csv'

In [ ]:
insurance = pd.read_csv(file_path_insurance_dataset)

In [ ]:
insurance.head()

In [ ]:
categorical = insurance.select_dtypes(include='object')
categorical

In [ ]:
insurance = pd.get_dummies(insurance, columns=list(categorical), drop_first=True)

In [ ]:
X = insurance.drop('charges', axis=1)
y = insurance['charges']

In [ ]:
X

In [ ]:
y

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(1)])

In [ ]:
model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.02),
        loss='mean_squared_error'
    )

In [ ]:
history = model.fit(
        X_train, y_train,
        epochs=1000,
        batch_size=23,
        validation_data=(X_test, y_test),
        verbose=1
    )

In [ ]:
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss (MSE): {test_loss:.4f}")

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# Plot the predicted vs actual values
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Charges')
plt.ylabel('Predicted Charges')
plt.title('Actual vs Predicted Charges')

# Plot the training loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()

plt.tight_layout()
plt.show()